# Zero-shot mutant prediction For Tgo-D4K

In [1]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch
import pandas as pd
from Bio import SeqIO
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore")
import os
os.environ["http_proxy"] = "http://l0:7890"
os.environ["https_proxy"] = "http://l0:7890"

def read_seq(seq_file):
    for record in SeqIO.parse(seq_file, "fasta"):
        return str(record.seq)

wild_type = f"../example_data/expr/tgod_d4k.fasta"
sequence = read_seq(wild_type)

all_mutants = []
for i, wt in enumerate(sequence):
    for aa in "ACDEFGHIKLMNPQRSTVWY":
        if aa != wt:
            mutant = f"{wt}{i+1}{aa}"
            all_mutants.append(mutant)
df = pd.DataFrame({"mutant": all_mutants})

In [2]:
base_model_path = "AI4Protein/tgo_d4k_"

for i in range(0, 5):
    model_path = f"{base_model_path}{i}"
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = AutoModelForMaskedLM.from_pretrained(model_path, trust_remote_code=True)
    tokenizer = AutoTokenizer.from_pretrained("AI4Protein/Prime_690M", trust_remote_code=True)
    model.eval()
    model = model.to(device)

    tokenized = tokenizer(sequence, return_tensors="pt").to(device)
    input_ids = tokenized["input_ids"]
    attention_mask = tokenized["attention_mask"]

    with torch.no_grad():
        output = model(input_ids, attention_mask=attention_mask)
        logits = output.logits[0, 1:-1, :].log_softmax(dim=-1)
    
    scores = []
    for mutant in tqdm(df["mutant"]):
        score = 0
        for sub_mutant in mutant.split(":"):
            wt, idx, mt = sub_mutant[0], int(sub_mutant[1:-1]) - 1, sub_mutant[-1]
            score += (logits[idx, tokenizer.get_vocab()[mt]] - logits[idx, tokenizer.get_vocab()[wt]]).item()
        scores.append(score)
    df[f"model_{i}"] = scores
    print(f"Model {i} done")

  0%|          | 0/14687 [00:00<?, ?it/s]

Model 0 done


  0%|          | 0/14687 [00:00<?, ?it/s]

Model 1 done


  0%|          | 0/14687 [00:00<?, ?it/s]

Model 2 done


  0%|          | 0/14687 [00:00<?, ?it/s]

Model 3 done


  0%|          | 0/14687 [00:00<?, ?it/s]

Model 4 done


In [5]:
aggregate_score = 0
for i in range(0, 5):
    # min-max
    df[f"model_{i}"] = (df[f"model_{i}"] - df[f"model_{i}"].min()) / (df[f"model_{i}"].max() - df[f"model_{i}"].min())
    aggregate_score += df[f"model_{i}"]
df["aggregate"] = aggregate_score
df = df.sort_values("aggregate", ascending=False)

In [8]:
df.head(50)

,mutant,model_0,model_1,model_2,model_3,model_4,aggregate
13165,I693W,1.000000,1.000000,1.000000,1.000000,1.000000,5.000000
8724,R460E,0.999637,0.999667,0.999650,0.999674,0.998928,4.997557
13590,P716G,0.990106,0.990094,0.990057,0.990060,0.989634,4.949951
9125,Y481G,0.982147,0.982201,0.982201,0.982243,0.981471,4.910263
10013,I528A,0.981692,0.981699,0.981674,0.981687,0.981146,4.907899
8933,I471E,0.977094,0.977117,0.977106,0.977127,0.976596,4.885040
4115,A217P,0.930181,0.930159,0.930130,0.930128,0.929996,4.650593
8443,F445K,0.925552,0.925589,0.925583,0.925616,0.924960,4.627300
8743,Q461E,0.901540,0.901505,0.901454,0.901447,0.900917,4.506864
14577,W768F,0.900632,0.900659,0.900633,0.900654,0.900180,4.502756
